In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter

In [2]:
edges = []
for j in range(9):
    for i in range(9):
        if i < j:
            edges.append((i,j))
#print(edges)

weights = [606.5,606.5,606.5,606.5,15.75,15.75,15.75,606.5,15.75,606.5,15.75,15.75,606.5,606.5,606.5,606.5,22.75,22.75,606.5,12,12,22.75,606.5,22.75,12,606.5,12,606.5,22.75,22.75,606.5,12,12,606.5,606.5,606.5]

graph = dict(zip(edges,weights))
print(graph) # all the zz terms and their weights

{(0, 1): 606.5, (0, 2): 606.5, (1, 2): 606.5, (0, 3): 606.5, (1, 3): 15.75, (2, 3): 15.75, (0, 4): 15.75, (1, 4): 606.5, (2, 4): 15.75, (3, 4): 606.5, (0, 5): 15.75, (1, 5): 15.75, (2, 5): 606.5, (3, 5): 606.5, (4, 5): 606.5, (0, 6): 606.5, (1, 6): 22.75, (2, 6): 22.75, (3, 6): 606.5, (4, 6): 12, (5, 6): 12, (0, 7): 22.75, (1, 7): 606.5, (2, 7): 22.75, (3, 7): 12, (4, 7): 606.5, (5, 7): 12, (6, 7): 606.5, (0, 8): 22.75, (1, 8): 22.75, (2, 8): 606.5, (3, 8): 12, (4, 8): 12, (5, 8): 606.5, (6, 8): 606.5, (7, 8): 606.5}


In [3]:
edges


[(0, 1),
 (0, 2),
 (1, 2),
 (0, 3),
 (1, 3),
 (2, 3),
 (0, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (0, 5),
 (1, 5),
 (2, 5),
 (3, 5),
 (4, 5),
 (0, 6),
 (1, 6),
 (2, 6),
 (3, 6),
 (4, 6),
 (5, 6),
 (0, 7),
 (1, 7),
 (2, 7),
 (3, 7),
 (4, 7),
 (5, 7),
 (6, 7),
 (0, 8),
 (1, 8),
 (2, 8),
 (3, 8),
 (4, 8),
 (5, 8),
 (6, 8),
 (7, 8)]

In [4]:
W = []
for i in range(len(edges)):
    W.append((edges[i][0],edges[i][1],weights[i]))
print(W)

[(0, 1, 606.5), (0, 2, 606.5), (1, 2, 606.5), (0, 3, 606.5), (1, 3, 15.75), (2, 3, 15.75), (0, 4, 15.75), (1, 4, 606.5), (2, 4, 15.75), (3, 4, 606.5), (0, 5, 15.75), (1, 5, 15.75), (2, 5, 606.5), (3, 5, 606.5), (4, 5, 606.5), (0, 6, 606.5), (1, 6, 22.75), (2, 6, 22.75), (3, 6, 606.5), (4, 6, 12), (5, 6, 12), (0, 7, 22.75), (1, 7, 606.5), (2, 7, 22.75), (3, 7, 12), (4, 7, 606.5), (5, 7, 12), (6, 7, 606.5), (0, 8, 22.75), (1, 8, 22.75), (2, 8, 606.5), (3, 8, 12), (4, 8, 12), (5, 8, 606.5), (6, 8, 606.5), (7, 8, 606.5)]


In [5]:
lattice_cite = [i for i in range(9)]
field_str = [-1290,-1290,-1290,-1268.5,-1268.5,-1268.5,-1282.5,-1282.5,-1282.5]
z_terms = dict(zip(lattice_cite,field_str))
print(z_terms) # all the z terms and their weights

{0: -1290, 1: -1290, 2: -1290, 3: -1268.5, 4: -1268.5, 5: -1268.5, 6: -1282.5, 7: -1282.5, 8: -1282.5}


In [11]:
len(z_terms.items())
param = [1,1]

In [7]:
def create_qaoa_circ_modified(W, theta, z_terms):
    """Creates a parametrized qaoa circuit
    Args:
        W: (tuple) containing edges and its respective weights 
        theta: (list) unitary parameters
        z_terms: (dict) containing single z terms and its respective weights
    Returns:
        (QuantumCircuit) qiskit circuit
    """
    nqubits = len(field_str)
    #print(nqubits)
    n_layers = len(theta)//2  # number of alternating unitaries
    beta = theta[:n_layers]
    gamma = theta[n_layers:]

    qc = QuantumCircuit(nqubits)

    # initial_state
    qc.h(range(nqubits))

    for layer_index in range(n_layers):
        #problem unitary
        for i in range(nqubits):
            qc.rz(2*gamma[layer_index]*field_str[i],i )
        for pair in W:  # pairs of nodes
            qc.rzz(2 * gamma[layer_index]*pair[2], pair[0], pair[1])
        # mixer unitary
        for qubit in range(nqubits):
            qc.rx(2 * beta[layer_index], qubit)

    qc.measure_all()
    return qc

In [30]:
#nv = number of vertices , cf = cost fun
# x = parameters
param = [1,1]
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
G.add_edges_from([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9)])

def Energy(cf, nv, x):
    Edges = [(i, j) for i, j in G]
    #qc_res = create_qaoa_circ(cf, nv, x)
    qc = create_qaoa_circ(G,param)
    backend = Aer.get_backend('statevector_simulator')
    job = execute(qc, backend,shots = 512)
    psi = job.result().get_statevector(qc, decimals=16)
    Exp_value = []
    for edges in G.edges():
        qc_res1 = create_qaoa_circ(G,param)
        for vertices in edges: # building the cost hamiltonian 
            qc_res1.z(vertices) # this is like adding the zz operator to each edge
            job1 = execute(qc_res1, backend)
            phi = job1.result().get_statevector(qc_res1, decimals=16)
            Exp_value.append(np.dot(np.transpose(np.conjugate(psizz)), phi))
    return np.real(np.sum(Exp_value))


In [31]:
# applying a cost hamiltonian H_c of TSP to QAOA anstaz 

def state_phi_zz(): # appending all the zz terms 
    Edges = [(i, j) for i, j in G]
    
    qc = create_qaoa_circ(G,param)
    backend = Aer.get_backend('statevector_simulator')
    job = execute(qc, backend)
    psi = job.result().get_statevector(qc, decimals=16)
    Exp_value = []
    for edges in graph.edges():
        psi_zz = create_qaoa_circ_modified(W, param, field_str)
        for vertices in edges: # building the cost hamiltonian
            psi_zz.z(vertices) # this is like adding the zz operator to each edge
            job1 = execute(psi_zz, backend)
            phi = job1.result().get_statevector(psi_zz, decimals=16)
            Exp_value.append(np.dot(np.transpose(np.conjugate(psi_zz)), phi))
    return np.real(np.sum(Exp_value))





In [28]:
def energy_zz():

    qc = create_qaoa_circ_modified(W, param, field_str)
    backend = Aer.get_backend('statevector_simulator')
    job = execute(qc, backend)
    
    psi = job.result().get_statevector(qc, decimals=8) # this is the state vector of the circuit
    
    Exp_value = []

    for i in range(len(edges)):
        
        qc_1 = create_qaoa_circ_modified(W, param, field_str)
        for vertices in edges[i]: # building the cost hamiltonian
            qc_1.z(vertices)
            backend = Aer.get_backend('statevector_simulator')
            job = execute(qc_1, backend)
            psi_zz = job.result().get_statevector(qc_1, decimals=8)
            Exp_value.append(0.5*list(graph.values())[i]*(np.dot(np.transpose(np.conjugate(psi_zz)), psi)))
        
    return np.real(np.sum(Exp_value))


In [29]:
# problem:
def energy_z():

    qc = create_qaoa_circ_modified(W, param, field_str)
    backend = Aer.get_backend('statevector_simulator')
    job = execute(qc, backend)
    
    psi1  = job.result().get_statevector(qc, decimals=8) # this is the state vector of the circuit
    
    Exp_value_z = []

    for i in range(len(field_str)):
        qc_2 = create_qaoa_circ_modified(W, param, field_str)
        qc_2.z(i)
        backend = Aer.get_backend('statevector_simulator')
        job = execute(qc_2, backend)
        psi_z = job.result().get_statevector(qc_2, decimals=8)
        
        Exp_value_z.append(field_str[i]*(np.dot(np.transpose(np.conjugate(psi_z)), psi1)))
        
    return np.real(np.sum(Exp_value_z))

In [31]:
energy = []
for i in range(10):
    energy.append(energy_zz() + energy_z())
print(np.sum(energy)/10)




-0.7874999977935604


In [23]:
print(energy)

[0.0, 1288.124998302748, 0.0, 0.0, 0.0, 0.0, -303.24999694801977, 0.0, 303.2499981217561, 0.0, 0.0, 589.12500607667, 0.0, 1268.500013084245, -303.2500019665204, 0.0, 617.8749991963258, 0.0, 0.0, -7.874999989756934, 0.0, 5.999999992195759, -5.374999993008702, 0.0, 0.0, 303.2500031279443, 0.0, 1268.499996877919, 0.0, -22.750000234660288, 309.25000318983274, 0.0, 11.375000064183254, 0.0, 0.0, 612.500006317777, 0.0, 303.25000153904097, 0.0, -1257.125002566773, 0.0, 0.0, -19.250000073865845, 322.50000065007464, 0.0, 0.0, -311.12500320917286, 5.999999992195759, -7.875000041336896, 0.0, 0.0, 0.0, 303.2499982949757, 0.0, 0.0, 0.0, 0.0, -322.49999958052206, 303.2499996055607, -303.2500031279443, -17.375000035023405, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 295.37500137041627, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1282.500006508887, 7.8750000158739155, 0.0, -303.2499991503456, 0.0, 0.0, 1282.4999920565613, 6.000000061888428, 0.0, 0.0, 0.0, 19.250000198558705, 303.2499996055607, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [22]:
offset = 7581.0
Energy = np.sum(energy)/100 + offset
print(Energy)

7671.415000273293


In [18]:
for i in edges:
    print(i)

(0, 1)
(0, 2)
(1, 2)
(0, 3)
(1, 3)
(2, 3)
(0, 4)
(1, 4)
(2, 4)
(3, 4)
(0, 5)
(1, 5)
(2, 5)
(3, 5)
(4, 5)
(0, 6)
(1, 6)
(2, 6)
(3, 6)
(4, 6)
(5, 6)
(0, 7)
(1, 7)
(2, 7)
(3, 7)
(4, 7)
(5, 7)
(6, 7)
(0, 8)
(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)


In [ ]:
def phi_zz():
    for edges in (list(graph.items())):
        qc_1 = create_qaoa_circ_modified(W, param, field_str)
        for vertices in edges[0]: # building the cost hamiltonian
            qc_1.z(vertices)
            backend = Aer.get_backend('statevector_simulator')
            job = execute(qc_1, backend)
            psi_zz = job.result().get_statevector(qc_1, decimals=8)